just standard gradcam i guess

next sticking in some bounding boxing


In [ ]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [8]:
# --- Load pre-trained model ---
model = models.resnet50(pretrained=True)
model.eval()

# --- Target layer for Grad-CAM ---
# target_layer = model.features[-1]  # last conv layer of VGG16
target_layer = model.layer4[-1]

# --- Image pre-processing ---
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])


C:\Users\hedge\.conda\envs\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\hedge/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


In [9]:
def load_image(img_path):
    img = Image.open(img_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)
    return img_tensor, np.array(img)

# --- Hook functions to capture activations and gradients ---
activations = None
gradients = None

def forward_hook(module, input, output):
    global activations
    activations = output.detach()

def backward_hook(module, grad_input, grad_output):
    global gradients
    gradients = grad_output[0].detach()

# --- Register hooks ---
target_layer.register_forward_hook(forward_hook)
target_layer.register_backward_hook(backward_hook)

In [10]:
# --- Grad-CAM function ---
def grad_cam(model, input_tensor, class_idx=None):
    global activations, gradients

    # Forward pass
    output = model(input_tensor)
    if class_idx is None:
        class_idx = torch.argmax(output)

    # Backward pass
    model.zero_grad()
    output[0, class_idx].backward()

    # Compute Grad-CAM
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])  # αᶜₖ
    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]
    
    cam = torch.sum(activations, dim=1).squeeze()
    cam = F.relu(cam)  # ReLU as described in the paper
    cam = cam - cam.min()
    cam = cam / cam.max()
    cam = cam.numpy()
    return cam

# --- Visualize heatmap ---
def overlay_heatmap(cam, original_image):
    cam = cv2.resize(cam, (original_image.shape[1], original_image.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    overlay = heatmap * 0.4 + original_image * 0.6
    return np.uint8(overlay)

In [14]:

# --- Run example ---
img_path = ''
input_tensor, original_image = load_image(img_path)
cam = grad_cam(model, input_tensor)
output_image = overlay_heatmap(cam, original_image)

plt.imshow(output_image[..., ::-1])  # Convert BGR to RGB
plt.axis('off')
plt.title("Grad-CAM Visualization")
plt.show()


PermissionError: [Errno 13] Permission denied: 'C:\\Workspace\\Code\\cs484proj'